In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"        # The model is pre-trained from a bunch of twitter posts/comments.
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
def senti_roberta(title):
    encoded_text = tokenizer(title, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    compound = scores_dict['roberta_neg'] + (scores_dict['roberta_neu'] * 2) + (scores_dict['roberta_pos'] *3) - 2
    return compound

In [ ]:
res = {}
for i, row in df_merged.iterrows():
    try:
        text = row['Title']
        date = row['Date']
        res[date] = senti_roberta(text)
        print(i)
    except RuntimeError:
        print(f'Broke for id {myid}')

In [ ]:
# Negative probability + (Neutral probability * 2) + Positive probability * 3) -2
comp = 0.000784 + (0.006838 * 2) + (0.992377 *3) - 2
comp